In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [2]:
ori_data = np.load('../Data/samples_24/energy_norm_truth.npy')
fake_data = np.load('../Data/ldm_fake_energy_112.npy')
iterations = 5

In [3]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.12601301687282201 

Iter 1:  context-fid = 0.11840981741444846 

Iter 2:  context-fid = 0.1234301642400388 

Iter 3:  context-fid = 0.10330994326462772 

Iter 4:  context-fid = 0.12612786681950172 

Final Score:  0.11945816172228774 ± 0.011862471474013907


In [4]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 1.2942825342683346 

Iter 1:  cross-correlation = 1.3368640025249836 

Iter 2:  cross-correlation = 1.3293583884469427 

Iter 3:  cross-correlation = 1.2487818383409603 

Iter 4:  cross-correlation = 1.423786771964518 

Final Score:  1.3266147071091479 ± 0.0801289727021006


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [6]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training: 100%|██████████| 2000/2000 [00:36<00:00, 55.50it/s]


Iter 0:  0.28252599543494805 , 0.7037788485924423 , 0.8612731422774537 



training: 100%|██████████| 2000/2000 [00:35<00:00, 56.06it/s]


Iter 1:  0.2556429114887142 , 0.7354805985290388 , 0.7758052244483895 



...

Iter 2:  0.2810043114379914 , 0.7344661425310677 , 0.8275424803449151 



training: 100%|██████████| 2000/2000 [00:36<00:00, 54.95it/s]


Iter 3:  0.2964747654070504 , 0.7727618564544763 , 0.8201876743596247 



training: 100%|██████████| 2000/2000 [00:35<00:00, 55.90it/s]

Iter 4:  0.2704793304590414 , 0.7060613745878772 , 0.8348972863302054 

sine:
Final Score:  0.2772254628455491 ± 0.018873985240567992



In [7]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training: 100%|██████████| 5000/5000 [01:13<00:00, 68.23it/s] 


0  epoch:  0.2508328946880133 



training: 100%|██████████| 5000/5000 [01:12<00:00, 68.49it/s] 


1  epoch:  0.25084343889911065 



training: 100%|██████████| 5000/5000 [01:06<00:00, 75.42it/s] 


2  epoch:  0.2510627455311388 



training: 100%|██████████| 5000/5000 [00:42<00:00, 117.80it/s]


3  epoch:  0.2508718406940236 



training: 100%|██████████| 5000/5000 [00:42<00:00, 116.79it/s]


4  epoch:  0.25095563339469595 

sine:
Final Score:  0.25091331064139644 ± 0.00011972652014996155

